In [ ]:
import os
import sys
import csv
import scipy as sp
import numpy as np
import pandas as pd
from os import listdir
from os import getcwd
from os.path import isfile, join

###################

#define a new path if desired, but should be the same as mypath above if new files were placed in working directory
mynewpath = "./Top_10_Individual_Mice"
#create a list of all of the Top10_Mouse_## dataframe names for iteration next
new_list_files = [f for f in listdir(mynewpath) if f.startswith("Top10_") and f.endswith(".csv") if isfile(join(mynewpath, f))]

#read in each dataframe, order by normalized Percent of Repertiore such that each list starts with the highest normalized frequency CDR3
#create a list of the actual dataframes for concatinating into one master list next
list_all_dataframes = []
print('Adding All Top10_Mouse_## dataframes to pandas dataframe list: list_all_dataframes')
for a, i in enumerate(new_list_files):
    data = pd.read_csv((mynewpath + '/' + new_list_files[a]), sep=',')
    data.sort_values(['Percent of Rep'], ascending = False)
    list_all_dataframes.append(data)

#add all of the ordered normalized dataframes from each individual mouse to a master dataframe
print('Concatinating to master pandas dataframe: all_mice')
all_mice = pd.concat(list_all_dataframes, ignore_index=True)

#define a function that creates a new list indicating the number of times a similar or identical sequence is found in separate mice
#ie. if similar sequences appear in the master list, they necessarily come from different mice, so this will count how many times it's found in separate mice
num_of_mice = []
def NumberofMice (s = all_mice.loc[0, 'TRAV'], t = all_mice.loc[0, 'TRAJ'], u = all_mice.loc[0, 'CDR3']):
    num_of_mice_n = 0
    for index, row in all_mice.iterrows():
        if row['TRAV'] == s and row['TRAJ'] == t:
            x = levenshteinDistance(u, row['CDR3'])
            if x <= 2:
                num_of_mice_n = num_of_mice_n + 1
    num_of_mice.append(num_of_mice_n)
    num_of_mice_n = 0

#append the list from above as a column in the master dataframe after iterating throught he master list with the NumberofMice function
length_all_mice = []
for index, row in all_mice.iterrows():
    length_all_mice.append(index)
for i in length_all_mice:
    NumberofMice (s = all_mice.loc[i, 'TRAV'], t = all_mice.loc[i, 'TRAJ'], u = all_mice.loc[i, 'CDR3'])
all_mice['Num of Mice Appearing'] = num_of_mice

#delete any sequences not appearing in 2 or more mice and are less than 10% of the total repertoire in the mouse
#sort the final master list by highest number of mice first, and highest frquency second
for index, row in all_mice.iterrows():
    if row['Num of Mice Appearing'] == 1 and row['Percent of Rep'] < 10.0:
        all_mice.drop(index, inplace=True)
all_mice = all_mice.sort_values(['Num of Mice Appearing', 'Percent of Rep'], ascending = [False, False])

#Print completed dataframe to a new .csv file for plotting in R
all_mice.to_csv('./Top_TCRs_Compiled_All_Mice.csv', sep = ',', encoding = 'utf-8', index = False)
print('Wrote all_mice as Top_TCRs_Compiled_All_Mice.csv in working directory')

#Run the R script using the new final list: Top_TCRs_Compiled_All_Mice.csv to get a visual plot

print("thanks for a great semester Stefano!")
print("Dave's project grade: NOT C++")
print("Dave's project grade: NOT B++")
print("Dave's project grade: A++!!!!!!!!!!!!!!!!!!")